#Prediksi Harga Saham 7 Hari ke Depan

#business understanding

Proyek ini bertujuan untuk menyediakan prediksi harga saham PT Bank Central Asia Tbk (BCA) untuk 7 hari ke depan secara akurat, sehingga dapat memberikan manfaat strategis bagi investor, trader, manajer portofolio, dan pengambil kebijakan di sektor keuangan.

Investor dapat memanfaatkan prediksi harga untuk merencanakan keputusan investasi dengan lebih baik, seperti menentukan waktu yang tepat untuk membeli atau menjual saham BCA. Dengan informasi prediksi yang akurat, investor dapat meminimalkan risiko dan mengoptimalkan pengembalian investasi mereka berdasarkan potensi pergerakan harga.

Trader dapat menggunakan prediksi ini sebagai bagian dari strategi trading jangka pendek. Prediksi harga membantu trader dalam mengambil posisi (buy atau sell) yang lebih terarah, meningkatkan peluang profit dalam kondisi pasar yang fluktuatif, serta mendukung pengambilan keputusan yang cepat dan data-driven.

Manajer portofolio dapat mengintegrasikan prediksi harga saham BCA ke dalam strategi pengelolaan portofolio mereka. Informasi ini membantu mereka mengalokasikan aset dengan lebih efisien, menyeimbangkan risiko portofolio, dan memberikan rekomendasi investasi yang lebih tepat kepada klien mereka.

#Data Understanding

data yang saya ambil merupakan data time series multivariate yaitu data harga saham BCA perhari yang diambil dari tanggal 02 Januari 2023 sampai 04 Desember 2024 pada provinsi jawa timur dengan jumlah data 456

In [1]:
import pandas as pd

# Membaca file Excel
file_path = '/content/sample_data/Data Historis BBCA.xlsx'  # Ganti dengan path file Excel Anda
df = pd.read_excel(file_path)

df.head()

,Tanggal,Terakhir,Pembukaan,Tertinggi,Terendah,Vol.,Perubahan%
0,02/01/2023,8.55,8.575,8.600,8.500,11.41,"0,00%"
1,03/01/2023,8.55,8.550,8.600,8.525,27.43,"0,00%"
2,04/01/2023,8.35,8.525,8.575,8.350,90.99,"-2,34%"
3,05/01/2023,8.25,8.350,8.375,8.150,128.84,"-1,20%"
4,06/01/2023,8.30,8.100,8.325,8.100,69.29,"0,61%"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456 entries, 0 to 455
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Tanggal     456 non-null    object 
 1   Terakhir    456 non-null    float64
 2   Pembukaan   456 non-null    float64
 3   Tertinggi   456 non-null    float64
 4   Terendah    456 non-null    float64
 5   Vol.        456 non-null    float64
 6   Perubahan%  456 non-null    object 
dtypes: float64(5), object(2)
memory usage: 25.1+ KB


**Deskripsi Data**:

Tanggal = merupakan tanggal diambilnya nilai pembukaan,tertinggi,terendah,vol,perubahan,terakhir

Terakhir = Harga terakhir saham yang diperdagangkan saat pasar tutup pada hari itu.

Pembukaan = Harga pertama saham yang diperdagangkan saat pasar dibuka pada hari itu

Tertinggi = Harga tertinggi saham yang dicapai selama periode perdagangan

Terendah =  Harga terendah saham yang dicapai selama periode perdagangan

Vol. = Jumlah total saham yang diperdagangkan selama periode tertentu

Perubahan% = Persentase perubahan harga saham dari harga penutupan hari sebelumnya ke harga penutupan hari ini

In [2]:
df.isna().sum()

,0
Tanggal,0
Terakhir,0
Pembukaan,0
Tertinggi,0
Terendah,0
Vol.,0
Perubahan%,0


menunjukkan bahwa tidak ada missing value

In [4]:
df.describe()

,Terakhir,Pembukaan,Tertinggi,Terendah,Vol.
count,456.000000,456.000000,456.000000,456.000000,456.000000
mean,9.438432,9.441064,9.510197,9.369134,77.693969
std,0.629773,0.627898,0.633883,0.618208,49.689246
min,8.175000,8.100000,8.300000,8.100000,11.410000
25%,8.975000,8.950000,9.025000,8.900000,52.997500
50%,9.287500,9.275000,9.325000,9.212500,67.040000
75%,10.000000,10.000000,10.050000,9.900000,89.367500
max,10.950000,10.900000,10.950000,10.775000,756.430000


#Data prepocessing

##fitur selection menurut korelasi antar fitur

In [45]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

file_path = '/content/sample_data/Data Historis BBCA.xlsx'  # Ganti dengan path file Excel Anda
df = pd.read_excel(file_path)

df = df.drop(['Tanggal','Perubahan%'], axis=1)
# Langkah 3: Tentukan kolom target
target_column = 'Terakhir'  # Ganti dengan nama kolom target Anda


# Buat objek LabelEncoder
label_encoder = LabelEncoder()

# Mengonversi kolom target
df[target_column] = label_encoder.fit_transform(df[target_column])

# Langkah 5: Hitung korelasi Pearson antara seluruh fitur dengan target
correlation_with_target = df.corr(method='pearson')[target_column]

# Menampilkan hasil korelasi
print("Korelasi Pearson antara seluruh fitur dengan target:")
print(correlation_with_target)

Korelasi Pearson antara seluruh fitur dengan target:
Terakhir     1.000000
Pembukaan    0.985663
Tertinggi    0.993986
Terendah     0.993419
Vol.        -0.010644
Name: Terakhir, dtype: float64


menurut hasil korelasi pearson hasil yang paling rendah hanyalah kolom volume maka kami akan menghapus kolom volume dan memakai kolom yang memiliki nilai korelasi tinggi

##normalisasi

In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib

# Membaca file Excel
file_path = '/content/sample_data/Data Historis BBCA.xlsx'  # Ganti dengan path file Excel Anda
df = pd.read_excel(file_path)

df = df.drop(['Vol.','Perubahan%'], axis=1)
# Inisialisasi MinMaxScaler
scaler = MinMaxScaler()

# Menampilkan nilai minimum dan maksimum sebelum normalisasi
min_values_before = df.min()
max_values_before = df.max()

print("Nilai minimum sebelum normalisasi:")
print(min_values_before)

print("\nNilai maksimum sebelum normalisasi:")
print(max_values_before)
# Melakukan normalisasi pada kolom numerik
kolom_numerik = df.select_dtypes(include=['float64', 'int64']).columns
df[kolom_numerik] = scaler.fit_transform(df[kolom_numerik])

scaler_file_path = '/content/sample_data/scaler.pkl'
joblib.dump(scaler, scaler_file_path)
# Menyimpan data yang sudah dinormalisasi ke file Excel baru
combined_file_path = '/content/sample_data/normalisasi.xlsx'
df.to_excel(combined_file_path, index=False)

Nilai minimum sebelum normalisasi:
Tanggal      01/02/2023
Terakhir          8.175
Pembukaan           8.1
Tertinggi           8.3
Terendah            8.1
dtype: object

Nilai maksimum sebelum normalisasi:
Tanggal      31/10/2024
Terakhir          10.95
Pembukaan          10.9
Tertinggi         10.95
Terendah         10.775
dtype: object


Disini kami menormalisasi data dulu agar nanti presentase errornya kecil dan kami menggunakan metode minmaxscaler untuk normalisasinya dimana data ini memiliki nilai minimum dan maksimum yang sudah tertera diatas

##mencari outlier

In [4]:
import pandas as pd
import numpy as np
from scipy import stats

# Langkah 1: Baca data dari Excel atau dataset
file_path = '/content/sample_data/normalisasi.xlsx'  # Ganti dengan path file Excel Anda
df = pd.read_excel(file_path)
# Langkah 2: Pilih kolom numerik yang ingin diuji outlier-nya
numerical_columns = ['Terakhir','Pembukaan','Tertinggi','Terendah']  # Ganti dengan kolom yang relevan

# Langkah 3: Hitung Z-Score untuk setiap fitur numerik
z_scores = np.abs(stats.zscore(df[numerical_columns]))

# Menentukan threshold, misalnya 3
threshold = 3

# Identifikasi outlier
outliers_zscore = np.where(z_scores > threshold)

# Menampilkan indeks dan kolom outlier
print("Indeks dan kolom yang mengandung outlier (Z-Score):")
print(outliers_zscore)

# Menampilkan data outlier
print("Data outlier berdasarkan Z-Score:")
print(df.iloc[outliers_zscore[0]])


Indeks dan kolom yang mengandung outlier (Z-Score):
(array([], dtype=int64), array([], dtype=int64))
Data outlier berdasarkan Z-Score:
Empty DataFrame
Columns: [Tanggal, Terakhir, Pembukaan, Tertinggi, Terendah]
Index: []


memperlihatkan bahwa data tidak memiliki outlier

##Pembagian Data

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the uploaded file
file_path = '/content/sample_data/normalisasi.xlsx'
data = pd.read_excel(file_path)

# Menyiapkan data (X, y)
X = data.drop(columns=['Terakhir', 'Tanggal'])
y = data['Terakhir']

# Menyimpan kolom tanggal untuk perbandingan nanti
dates = data['Tanggal']

# Split data into training and testing sets
X_train, X_test, y_train, y_test, train_dates, test_dates = train_test_split(
    X, y, dates, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")
# Simpan data yang dipisah ke dalam satu file Excel dengan sheet berbeda
output_file = '/content/sample_data/bagi.xlsx'
with pd.ExcelWriter(output_file) as writer:
    train_dates,X_train.to_excel(writer, sheet_name='X_train', index=False)
    test_dates,X_test.to_excel(writer, sheet_name='X_test', index=False)
    train_dates,y_train.to_excel(writer, sheet_name='y_train', index=False)
    test_dates,y_test.to_excel(writer, sheet_name='y_test', index=False)


output_file


Training set size: 364
Testing set size: 92


'/content/sample_data/bagi.xlsx'

membagi data train sebanyak 364 dan tes 92

#Data modeling

##Ensamble Bagging

In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib

scaler_file_path = '/content/sample_data/scaler.pkl'
scaler = joblib.load(scaler_file_path)
# Load the uploaded file
file_path = '/content/sample_data/bagis.xlsx'
data = pd.read_excel(file_path)

# Model dasar menggunakan Linear Regression
base_model = LinearRegression()

# Membuat model Bagging dengan estimator dasar Linear Regression
bagging_model = BaggingRegressor(estimator=base_model, n_estimators=10, random_state=42)

# Melatih model
bagging_model.fit(X_train, y_train)

# Melakukan prediksi pada data uji
predictions = bagging_model.predict(X_test)

# Menghitung metrik evaluasi
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
mape = mean_absolute_percentage_error(y_test, predictions)

# Menampilkan hasil evaluasi
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape * 100:.2f}%")

# Input data baru untuk prediksi
new_data = np.array([[10.5, 10, 10.1]])  # Input baru: Open, High, Low

# Konversi input baru menjadi DataFrame dengan nama fitur yang sama seperti X_train
new_data_df = pd.DataFrame(new_data)

normalisasi = pd.DataFrame(
    scaler.transform(new_data_df),  # Menggunakan scaler yang dimuat
    columns=new_data_df.columns
)  # Menggunakan scaler yang dimuat
# Prediksi multi-step untuk 7 hari ke depan
n_steps = 7
multi_step_predictions = []

# Melakukan prediksi secara berurutan untuk setiap langkah
current_input = normalisasi
for step in range(n_steps):
    # Melakukan prediksi untuk langkah berikutnya
    prediction = bagging_model.predict(current_input)
    multi_step_predictions.append(prediction[0])

    # Update current_input dengan hasil prediksi untuk langkah berikutnya
    # Formatkan ulang input agar tetap DataFrame dengan nama kolom yang sama
    current_input = pd.DataFrame(
        np.hstack((np.array([[prediction[0]]]), current_input.values[:, :-1])),
        columns=X_train.columns
    )

# Tampilkan hasil prediksi
for i, pred in enumerate(multi_step_predictions, 1):
    print(f"Prediksi hari ke-{i}: {pred}")

# Denormalisasi hasil prediksi harga untuk semua data dalam array
min_harga = 8.175  # Nilai minimum harga dalam dataset asli
max_harga = 10.95  # Nilai maksimum harga dalam dataset asli

# Denormalisasi nilai prediksi setiap hari
denormalized_predictions = [
    pred * (max_harga - min_harga) + min_harga for pred in multi_step_predictions
]

# Tampilkan hasil prediksi denormalisasi untuk setiap hari ke depan
for i, denormalized_price in enumerate(denormalized_predictions, 1):
    print(f"Prediksi denormalisasi hari ke-{i}: {denormalized_price:,.2f}")



MAE: 0.013538276232711286
RMSE: 0.01740884595168998
MAPE: 5.15%
Prediksi hari ke-1: 0.5649650169104838
Prediksi hari ke-2: 0.857227419724229
Prediksi hari ke-3: 0.5930196688729086
Prediksi hari ke-4: 0.7746014300754163
Prediksi hari ke-5: 0.6737776010339479
Prediksi hari ke-6: 0.6741067499313635
Prediksi hari ke-7: 0.741677479826693
Prediksi denormalisasi hari ke-1: 9.74
Prediksi denormalisasi hari ke-2: 10.55
Prediksi denormalisasi hari ke-3: 9.82
Prediksi denormalisasi hari ke-4: 10.32
Prediksi denormalisasi hari ke-5: 10.04
Prediksi denormalisasi hari ke-6: 10.05
Prediksi denormalisasi hari ke-7: 10.23


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but BaggingRegressor was fitted with feature names
  warnings.warn(


pada data ini model yang paling bagus presentase errornya adalah dengan menggunakan metode ensamble bagging

##Decision Tree Regressor

In [61]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Path file (ganti dengan path file Anda sendiri)
file_path = '/content/sample_data/bagi.xlsx'

# Memuat data
data = pd.read_excel(file_path)

# Mendefinisikan dan melatih model Decision Tree
dt_model = DecisionTreeRegressor(random_state=42, max_depth=5)  # Parameter dapat disesuaikan
dt_model.fit(X_train, y_train)

# Memprediksi pada set pengujian
y_pred = dt_model.predict(X_test)

# Menghitung dan mencetak metrik evaluasi
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)

# Menampilkan hasil evaluasi
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape * 100:.2f}%")


# --- Membandingkan prediksi dengan data aktual ---
# Membuat DataFrame untuk nilai aktual vs prediksi
comparison_df = pd.DataFrame({
    "tanggal": X_test.index,
    "Actual": y_test,
    "Predicted": y_pred
})

# Mengurutkan DataFrame berdasarkan tanggal
comparison_df = comparison_df.sort_values(by="tanggal").reset_index(drop=True)

print("\nHarga Aktual vs Prediksi:")
print(comparison_df)


MAE: 0.019136356958381954
RMSE: 0.022951259922704297
MAPE: 10.68%

Harga Aktual vs Prediksi:
    tanggal    Actual  Predicted
0         0  0.189189   0.210210
1         1  0.810811   0.827542
2         2  0.315315   0.307748
3         3  0.468468   0.423423
4         4  0.468468   0.457658
..      ...       ...        ...
80       80  0.351351   0.351624
81       81  0.324324   0.307748
82       82  0.306306   0.351624
83       83  0.810811   0.795796
84       84  0.774775   0.827542

[85 rows x 3 columns]


##Random Forest Regressor

In [62]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Path file (ganti dengan path file Anda sendiri)
file_path = '/content/sample_data/bagi.xlsx'

# Memuat data
data = pd.read_excel(file_path)

# Mendefinisikan dan melatih model Decision Tree
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Menghitung dan mencetak metrik evaluasi
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape * 100:.2f}%")



# --- Membandingkan prediksi dengan data aktual ---
# Membuat DataFrame untuk nilai aktual vs prediksi
comparison_df = pd.DataFrame({
    "tanggal": X_test.index,
    "Actual": y_test,
    "Predicted": y_pred
})

# Mengurutkan DataFrame berdasarkan tanggal
comparison_df = comparison_df.sort_values(by="tanggal").reset_index(drop=True)

print("\nHarga Aktual vs Prediksi:")
print(comparison_df)


MAE: 0.017492732228026324
RMSE: 0.020987690316540495
MAPE: 11.08%

Harga Aktual vs Prediksi:
    tanggal    Actual  Predicted
0         0  0.189189   0.203610
1         1  0.810811   0.838649
2         2  0.315315   0.309755
3         3  0.468468   0.452703
4         4  0.468468   0.478018
..      ...       ...        ...
80       80  0.351351   0.359865
81       81  0.324324   0.306471
82       82  0.306306   0.336261
83       83  0.810811   0.791712
84       84  0.774775   0.827387

[85 rows x 3 columns]


##XGBRegressor

In [63]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Path file (ganti dengan path file Anda sendiri)
file_path = '/content/sample_data/bagi.xlsx'

# Memuat data
data = pd.read_excel(file_path)

# Mendefinisikan dan melatih model Decision Tree
model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Menghitung dan mencetak metrik evaluasi
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape * 100:.2f}%")



# --- Membandingkan prediksi dengan data aktual ---
# Membuat DataFrame untuk nilai aktual vs prediksi
comparison_df = pd.DataFrame({
    "tanggal": X_test.index,
    "Actual": y_test,
    "Predicted": y_pred
})

# Mengurutkan DataFrame berdasarkan tanggal
comparison_df = comparison_df.sort_values(by="tanggal").reset_index(drop=True)

print("\nHarga Aktual vs Prediksi:")
print(comparison_df)


MAE: 0.016705553014883624
RMSE: 0.01986092097159439
MAPE: 9.67%

Harga Aktual vs Prediksi:
    tanggal    Actual  Predicted
0         0  0.189189   0.204524
1         1  0.810811   0.843569
2         2  0.315315   0.311324
3         3  0.468468   0.455628
4         4  0.468468   0.461630
..      ...       ...        ...
80       80  0.351351   0.367882
81       81  0.324324   0.307608
82       82  0.306306   0.324903
83       83  0.810811   0.781235
84       84  0.774775   0.812512

[85 rows x 3 columns]
